## Advanced Himawari Imager (AHI) Demo

In this example we will learn how to read in and visualise data from the Advanced Himawari Imager.

In [ ]:
# Ignore warning and debug messages
# NB: If things go wrong, you can turn the debugger on and turn the warnings filter off.
import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")
from satpy.utils import debug_off; debug_off()


from satpy import Scene
from satpy import available_readers
from datetime import datetime
from glob import glob

Whenever we use SatPy, we will always use the Scene object to read in various satellite datasets. Xarray is working under the hood to interpret the various file formats that satellite data comes in.

When SatPy is first installed, not all of the supported satellite data 'readers' are included. So we can first look at what comes from a standard install of SatPy using the `available_readers()` object (imported above).

In [ ]:
print(available_readers())

Above we can see that `available_readers()` returns a list of the available satellite instrument readers that your SatPy install can handle. For reference, all of the supported readers can be viewed in the table from the SatPy user guide: https://satpy.readthedocs.io/en/stable/index.html#reader-table.

Once we have the data downloaded, we tell SatPy where to find the appropriate files with the helper function `find_files_and_readers`

In [ ]:
from satpy import find_files_and_readers

path_ahi = './data/ahi/'
files = find_files_and_readers(start_time=datetime(2023, 4, 17, 23), 
                               end_time=datetime(2023, 4, 17, 23, 10),
                               reader='ahi_hsd',
                               base_dir=path_ahi)

# Check the list of files
print(files)

The filename tells us a bit about the data we're looking at. For example, `H09` means it's Himawari-9, `20230417_2300` is the time and date in `YYYYmmdd_HHMM` format, `B01` means it's band (or channel) `R10` means 1 km resolution and `S0110` means the file corresponds to the first segment of the 10 segments that make up the full disk. Once we have the file names, we can load the data into the Scene object:

In [ ]:
# Load data into Scene object
scn = Scene(files)

We can print out some initial information regarding the various channels we have available with:

In [ ]:
scn.available_dataset_ids()

We can also just list the bands in a list with:

In [ ]:
scn.available_dataset_names()

And we can check the available composites with:

In [ ]:
scn.available_composite_names()

If we want to interrogate and process the data, we must first load the channels we are interested in using the Scene object. We can quickly load all of the channels with:

In [ ]:
scn.load(scn.available_dataset_names())

Now that we have the data loaded, we can view more metadata using the print function:

In [ ]:
print(scn)

If we want to view the data, we can use the `show()` method:

In [ ]:
scn.show('B01')

Say we wanted to look at different channels side-by-side we could do

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

ax1.imshow(scn['B01'], cmap='Greys_r')
ax1.set_title('B01')

ax2.imshow(scn['B14'], cmap='Greys')
ax2.set_title('B14')

ax3.imshow(scn['B10'], cmap='Greys')
ax3.set_title('B10')

fig.tight_layout()
plt.show()


Now if we want to make a pretty picture, we can call a composite. Let's first try the `true_color_raw` composite, which doesn't apply any atmospheric correction or colour enhancments:

In [ ]:
scn.load(['true_color_raw'])
# Now that the data must be resampled because the R, G and B channels are sampled at different spatial resolutions.
lcn = scn.resample(scn.coarsest_area())

In [ ]:
lcn.show('true_color_raw')

Now we can look at a specific area using a series of pre-defined regional areas that are built into SatPy. Let's look at the `true_color` composite (which includes an atmospheric correction and colour enhancement) over Australia.

In [ ]:
scn.load(['true_color'])

The pre-defined areas can be found in your `/etc/areas.yaml` file.

In [ ]:
! cat /Users/apra0062/mambaforge/envs/satpy-env/lib/python3.11/site-packages/satpy/etc/areas.yaml

In [ ]:
lcn = scn.resample('australia')

In [ ]:
lcn.show('true_color')

Now let's say we're interested in a particular area, not included in the SatPy `areas.yaml` file. Often we only know the latitude/longitude extents. Pyresample uses the PROJ conventions to define map projections. More information can be found here: https://proj.org/. We can create a resampling object, known as an `area_definition`, which we can use to resample the AHI full disk to. For example:

In [ ]:
from pyresample import create_area_def

# Let's look at a region that covers Macquarie Isalnd
extent = [158.6, 159.2, -54.8, -54.4]
area_def = create_area_def('Macquarie Island', "+proj=eqc", units="degrees",
                           area_extent=(extent[0], extent[2], extent[1], extent[3]), resolution=0.0001)
# Resample to custom area def
lcn = scn.resample(area_def)
lcn.show('true_color')

Now we can see Macquarie Island just looks like a collection of blurred pixels in the AHI data. This is because the spatial resolution of the visible pixels is 1 km. In the MODIS and MSI examples we'll see how this improves with better spatial resolutions from the polar orbiting sensors.

## Exercise
AHI covers an entire Earth's disk. Can you reproject the AHI data to zoom into another regional area of interest? Maybe Japan or how about over South Australia?